# Erstellen einer Umsatzprognose
© Thomas Robert Holy 2019
<br>
Version 0.0.2
<br><br>
Visit me on GitHub: https://github.com/trh0ly

## Grundlegende Einstellungen

### Package Import

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import math

### Optikeinstellungen

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.width', 350)
plt.rcParams['figure.figsize'] = (12, 6) # macht die Plots größer

## Datensatz und Datensatzmanipulation

In [ ]:
#-----------------------
# Amazon

quartals = ["Q1'07", "Q2'07", "Q3'07", "Q4'07", "Q1'08", "Q2'08", "Q3'08", "Q4'08", "Q1'09", "Q2'09",
            "Q3'09", "Q4'09", "Q1'10", "Q2'10", "Q3'10", "Q4'10", "Q1'11", "Q2'11", "Q3'11", "Q4'11",
            "Q1'12", "Q2'12", "Q3'12", "Q4'12", "Q1'13", "Q2'13", "Q3'13", "Q4'13", "Q1'14", "Q2'14",
            "Q3'14", "Q4'14", "Q1'15", "Q2'15", "Q3'15", "Q4'15", "Q1'16", "Q2'16", "Q3'16", "Q4'16",
            "Q1'17", "Q2'17", "Q3'17", "Q4'17", "Q1'18", "Q2'18", "Q3'18", "Q4'18", "Q1'19", "Q2'19"]

revenues = [3.02, 2.89, 3.26, 5.67, 4.14, 4.06, 4.26, 6.7, 4.89, 4.65, 5.45, 9.52, 7.13, 6.57, 7.56,
            12.95, 9.86, 9.91, 10.88, 17.43, 13.19, 12.83, 13.81, 21.27, 16.07, 15.7, 17.09, 25.59,
            19.74, 19.34, 20.58, 29.33, 22.72, 23.19, 25.36, 35.75, 29.13, 30.4, 32.71, 43.74, 35.71,
            37.96, 43.74, 60.5, 51.04, 52.89, 56.58, 72.38, 59.7, 63.4]

print(len(quartals))
print(len(revenues))

#-----------------------
# DataFrame erstellen

rev_DF = pd.DataFrame({'Revs':revenues, 'Quart':range(1,len(revenues) + 1)})
rev_DF.head()

### DataFrame plotten

In [ ]:
rev_DF.Revs.plot(figsize=(12,8), title= 'Revenues', fontsize=10)

## Definition einige Hilfsfunktionen

### Moving-Average

In [ ]:
def moving_average(k, array, weights=None):
    
    # Keine Gewichtung von Perioden
    if weights == None:
        z_liste = [] # Leere Liste für MA
        # Bestimmung MA als Mittelwert aller Elemente um i mit der Reichweite k
        for i in range(k, len(array) - k):
            x = (i - k) # Vorgängerposition
            y = (i + k + 1) # Nachfolgerposition
            z = np.mean(array[x:y]) # Mittelwert von Vorgängerposition bis Nachfolgerposition
            z_liste.append(z) # Wert der Liste anfügen
            
    # Gewichtung von Perioden
    if weights != None:
        z_liste, temp_liste = [], [] # Leere Liste für MA und Zwischenschritt
        # Bestimmung MA als Mittelwert aller Elemente um i mit der Reichweite k
        for i in range(k, len(array) - k):
            x = (i - k) # Vorgängerposition
            y = (i + k + 1) # Nachfolgerposition
            temp = array[x:y] # Zwischenschritt welcher alle Elementen abspeichert
            temp_liste.append(temp) # Wert der Liste anfügen (Main-Array)
            
        # Für jedes Main-Array..
        for i in temp_liste:
            # ..Gewichtung der Elemete mit dazugehörigem Gewicht
            for j in range(0, len(i)):
                i[j] = i[j] * weights[j]
            
        # Bildung Mittelwert der gewichteten Elemente
        for i in temp_liste:
            i = np.mean(i)
            z_liste.append(i)
                   
    return z_liste    

### $\Delta t, \Delta z$ und $\Delta t \Delta z$

In [ ]:
#-----------------------
# Delta t

def delta_t(t_array):

    mean_t = np.mean(t_array) # Mittelwert der Peridoen bestimmen
    
    delta_t_list = []
    # Delta t als Differenz von t und dem Mittelwert von allen t-Werten bestimmen
    for i in range(0, len(t_array)):
        delta_t = t_array[i] - mean_t # Differenz bilden
        delta_t_list.append(delta_t) # Wert der Liste anfügen
        
    delta_t_power_liste = [] # Leere Liste anlegen
    # Delta t^2 für jedes t bestimmen
    for i in delta_t_list:
        delta_t_power = np.power(i,2) # t quadrieren
        delta_t_power_liste.append(delta_t_power) # Wert der Liste anfügen
        
    # Summe der quadrierten ts bestimmen
    sum_delta_t_power_liste = np.sum(delta_t_power_liste)
    
    return delta_t_list, delta_t_power_liste, sum_delta_t_power_liste

#-----------------------
# Delta z

def delta_z(z_array, exp=False):
    
    mean_z = np.mean(z_array) # Mittelwert der beobachteten Werte bestimmen
    
    # Wenn ein exponentieller Trend vorliegt
    if exp == False:
        delta_z_list = [] # Leere Liste anlegen
        # Delta z als Differenz von z und dem Mittelwert von allen z-Werten bestimmen
        for i in range(0, len(z_array)):
            delta_z = z_array[i] - mean_z # Differenz bilden
            delta_z_list.append(delta_z) # Wert der Liste anfüge
            
    # Wenn kein exponentieller Trend vorliegt
    if exp == True:
        delta_z_list = [] # Leere Liste anlegen
        
        # Delta z^2 für jedes z bestimmen
        for i in range(0, len(z_array)):
            delta_z = np.log(z_array[i]) - np.log(mean_z) # Differenz bilden
            delta_z_list.append(delta_z) # Wert der Liste anfüge
                 
    return delta_z_list

#-----------------------
# Delta t Delta z

def delta_z__delta_t(delta_t_list, delta_z_list):
    
    delta_zt_liste = []  # Leere Liste anlegen
    # delta t mit delta z multiplizieren
    for i in range(0, len(delta_t_list)):
        delta_zt = delta_t_list[i] * delta_z_list[i] # delta t mit delta z multiplizieren
        delta_zt_liste.append(delta_zt) # Wert der Liste anfüge        
    
    # Summe der multiplizierten Werte bestimmen
    sum_delta_zt_liste = np.sum(delta_zt_liste)
    
    return delta_zt_liste, sum_delta_zt_liste

### Berechnung $a$ und $b$

In [ ]:
def calc_ab(z_array, t_array, delta_zt_liste, sum_delta_t_power_liste):
    
    # Mittelwerte bestimmen
    mean_z = np.mean(z_array)
    mean_t = np.mean(t_array)
       
    # Parameter a und bestimmen
    b = sum_delta_zt_liste / sum_delta_t_power_liste # Steigung berechnen
    a = mean_z - b * mean_t # Intercept berechnen
    
    return a, b

### Prognose

In [ ]:
def predict(a, b, array, exp=False):
    
    # Wenn kein exponentieller Trend vorliegt
    if exp == False:
        prediction_list = [] # Leere Liste anlegen
        # Berechnung des zu prognostizierenden Wertes
        for i in range(0, len(array)):
            prediction = a + b * array[i] # Prognostizierter Wert = Intercept + Steigung * Zeitreihenwert
            prediction_list.append(prediction) # Wert der Liste anfüge  
            
    # Wenn ein exponentieller Trend vorliegt
    if exp == True:
        prediction_list = [] # Leere Liste anlegen
         # Berechnung des zu prognostizierenden Wertes
        for i in range(0, len(array)):
            prediction = np.exp(a) * math.exp((b * array[i])) # Prognostizierter Wert = Intercept + Steigung * Zeitreihenwert
            prediction_list.append(prediction) # Wert der Liste anfüge  
        
    return prediction_list

### Bestimmtheitsmaß

In [ ]:
def r_power(z_array, prediction_list):
    
    mean_z = np.mean(z_array) # Mittelwert bestimmen
    
    a_list, b_list = [], [] # Leere Listen anlegen
    # Berechnung der quadrierten Abweichungen..
    for i in range(0, len(z_array)):
        # Prognostizierte Werte
        a = np.power((prediction_list[i] - mean_z), 2)
        a_list.append(a)
        
        # Tatsächlich beobachtete Werte
        b = np.power((z_array[i] - mean_z), 2)
        b_list.append(b)
        
    # Summen der beiden Listen ermitteln 
    sum_a_list = np.sum(a_list)
    sum_b_list = np.sum(b_list)
    
    r_power = sum_a_list / sum_b_list # Bestimmtheitsmaß berechnen
    
    return r_power

## Schrittweise Berechnung - Einfache lineare Regression

### $\Delta t$ und Summe $\Delta t$

In [ ]:
t_array = rev_DF.Quart.values.tolist()

delta_t_list, delta_t_power_liste, sum_delta_t_power_liste = delta_t(t_array)
#print(delta_t_list, delta_t_power_liste, sum_delta_t_power_liste)

### $\Delta z$ und Summe $\Delta z$

In [ ]:
z_array = rev_DF.Revs.values.tolist()

delta_z_list = delta_z(z_array)
#delta_z_list

### $\Delta t \Delta z$ und Summe $\Delta t \Delta z$

In [ ]:
delta_zt_liste, sum_delta_zt_liste = delta_z__delta_t(delta_t_list, delta_z_list)
#print(delta_zt_liste, sum_delta_zt_liste)

### Parameter $a$ und $b$

In [ ]:
a, b = calc_ab(z_array, t_array, delta_zt_liste, sum_delta_t_power_liste)
print(b, a)

### Prognosewerte für die 1. bis letzte Periode

In [ ]:
#prediction_for_list = range(1,13)
prediction_for_list = range(1,len(revenues) + 1)

prediction_list = predict(a, b, prediction_for_list)
#prediction_list

### DataFrame-Erweiterung und plotten der Prognosewerte

In [ ]:
rev_DF['Pred'] = prediction_list

In [ ]:
orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(rev_DF.Pred, color='red', label='Pred')
plt.legend(loc='best')
plt.title('Original vs. Pred')
plt.show()

### Bestimmtheitsmaß berechnen

In [ ]:
r = r_power(z_array, prediction_list)
r

## Schrittweise Berechnung unter Berücksichtigung nicht-linearer Zusammenhänge

### $\Delta t$ und Summe $\Delta t$

In [ ]:
t_array = rev_DF.Quart.values.tolist()

delta_t_list, delta_t_power_liste, sum_delta_t_power_liste = delta_t(t_array)
#print(delta_t_list, delta_t_power_liste, sum_delta_t_power_liste)

### $\Delta z$ und Summe $\Delta z$

In [ ]:
z_array = rev_DF.Revs.values.tolist()
#z_array = np.log(z_array)

delta_z_list = delta_z(z_array, exp=True)
#delta_z_list

### $\Delta t \Delta z$ und Summe $\Delta t \Delta z$

In [ ]:
delta_zt_liste, sum_delta_zt_liste = delta_z__delta_t(delta_t_list, delta_z_list)
#print(delta_zt_liste, sum_delta_zt_liste)

### Parameter $a$ und $b$

In [ ]:
a, b = calc_ab(np.log(z_array), t_array, delta_zt_liste, sum_delta_t_power_liste)
print(b, a)

### Prognosewerte für die 1. bis letze Periode

In [ ]:
#prediction_for_list = range(1,13)
prediction_for_list = range(1,len(revenues) + 1)
z_array = z_array

prediction_list_exp = predict(a, b, prediction_for_list, exp=True)
#prediction_list_exp

### DataFrame-Erweiterung und plotten der Prognosewerte

In [ ]:
rev_DF['Pred_exp'] = prediction_list_exp

In [ ]:
orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(rev_DF.Pred_exp, color='red', label='Pred')
plt.legend(loc='best')
plt.title('Original vs. Pred_exp')
plt.show()

### Bestimmtheitsmaß berechnen

In [ ]:
r = r_power(z_array, prediction_list_exp)
r

## Weitere Verbesserungen
### Definition weiterer Hilfsfunktionen
#### Berechnung des Residuums zwischen Moving-Average und Beobachtung

In [ ]:
def calc_residual(rev, moving_average_list):
    
    # Liste Revenues auf Länge des MA kürzen
    cut_len = len(rev) - len(moving_average_list) # Differenz der Längen bestimmen
    start = int(cut_len / 2) # Startwert bestimmen
    end = int(len(rev) - cut_len / 2) # endwert bestimmen    
    rev = rev[start:end] # Liste kürzen
    
    residual_list = [] # Leere liste anlegen
    # Abweichung zwischen beobachteten Wert und MA berechnen
    for i in range(0, len(rev)):
        residuum = rev[i] - moving_average_list[i] # differenz berechnen
        residual_list.append(residuum) # Wert der Liste anfügen
        
    return residual_list    

#### Berechnung mittlere quartalsweise Sainsonkopmponente

In [ ]:
def get_seasonal_components(residual_list):
    
    quartal_1_list = [] # Leere Liste anlegen
    # Jedes 4. Quartal ausgehend vom ersten Quartal
    # in der Liste speichern
    for i in range(0, len(residual_list), 4):
        quartal_1_list.append(residual_list[i])
     
    quartal_2_list = [] # Leere Liste anlegen
    # Jedes 4. Quartal ausgehend vom zweiten Quartal
    # in der Liste speichern
    for i in range(1, len(residual_list), 4):
        quartal_2_list.append(residual_list[i])
     
    quartal_3_list = [] # Leere Liste anlegen
    # Jedes 4. Quartal ausgehend vom dritten Quartal
    # in der Liste speichern
    for i in range(2, len(residual_list), 4):
        quartal_3_list.append(residual_list[i])
     
    quartal_4_list = [] # Leere Liste anlegen
    # Jedes 4. Quartal ausgehend vom vierte Quartal
    # in der Liste speichern
    for i in range(3, len(residual_list), 4):
        quartal_4_list.append(residual_list[i])
        
    # Mittelwerte der Listen berechnen um durchschnittliche
    # Saisonkomponente zu bestimmen
    seasonal_component_quartal_1 = np.mean(quartal_1_list)
    seasonal_component_quartal_2 = np.mean(quartal_2_list)
    seasonal_component_quartal_3 = np.mean(quartal_3_list)
    seasonal_component_quartal_4 = np.mean(quartal_4_list)
    
    # Liste mit Saisonkomponenten der Länge der residual_list angleichen
    seasonal_components_list = [seasonal_component_quartal_1, seasonal_component_quartal_2,
                               seasonal_component_quartal_3, seasonal_component_quartal_4] * int(len(residual_list) / 4)
    
    return seasonal_components_list

#### Verbindung Saisonkomponente und MA-Komponente

In [ ]:
def add_saison(prediction_list_MA, seasonal_components):
    
    ma_saison_list = [] # Leere Liste anlegen
    # Saisonkomponente und MA zusammenfügen
    for i in range(0, len(prediction_list_MA)):
        x = prediction_list_MA[i] + seasonal_components[i] # Saisonkomponente und MA addieren
        ma_saison_list.append(x) # Wert der Liste anfügen
        
    return ma_saison_list        

### Moving-Average als Trend

In [ ]:
k = 3

moving_average_list = moving_average(k, revenues, weights=[1,0.75,1,1,1,0.6,1])
#moving_average_list

#### DataFrame-Erweiterung und plotten der Prognosewerte auf Basis des MA

In [ ]:
test_DF = pd.DataFrame({'Quartal':range(k,len(moving_average_list) + k ), 'Revs':moving_average_list})
#test_DF.head()

In [ ]:
orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(test_DF.Quartal, test_DF.Revs, color='red', label='Moving Average')
plt.legend(loc='best')
plt.title('Original vs. Moving Average')
plt.show()

#### $\Delta t$ und Summe $\Delta t$

In [ ]:
#t_array = test_DF.Quart.values.tolist()
t_array = range(1, len(moving_average_list))

delta_t_list, delta_t_power_liste, sum_delta_t_power_liste = delta_t(t_array)
#print(delta_t_list, delta_t_power_liste, sum_delta_t_power_liste)

#### $\Delta z$ und Summe $\Delta z$

In [ ]:
z_array = test_DF.Revs.values.tolist()
#z_array = np.log(z_array)

delta_z_list = delta_z(z_array, exp=True)
#delta_z_list

#### $\Delta t \Delta z$ und Summe $\Delta t \Delta z$

In [ ]:
delta_zt_liste, sum_delta_zt_liste = delta_z__delta_t(delta_t_list, delta_z_list)
#print(delta_zt_liste, sum_delta_zt_liste)

#### Parameter $a$ und $b$

In [ ]:
a, b = calc_ab(np.log(z_array), t_array, delta_zt_liste, sum_delta_t_power_liste)
print(b, a)

#### Prognosewerte für die 1. bis letze Periode

In [ ]:
#prediction_for_list = range(1,13)
prediction_for_list = range(1,len(test_DF.Revs) + 1)

prediction_list_MA = predict(a, b, prediction_for_list, exp=True)
#prediction_list_MA

#### DataFrame-Erweiterung und plotten der Prognosewerte  auf Basis MA

In [ ]:
test_DF['Pred_MA'] = prediction_list_MA

In [ ]:
orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(test_DF.Quartal, test_DF.Pred_MA, color='red', label='Pred')
plt.legend(loc='best')
plt.title('Original vs. Pred_MA')
plt.show()

### Hinzufügen der Saisonkomponente

#### Residuen berechnen

In [ ]:
residual_list = calc_residual(revenues, moving_average_list)
#residual_list

#### Saisonkomponenten berechnen

In [ ]:
seasonal_components = get_seasonal_components(residual_list)
#seasonal_components

#### Saisonkomponente auf Moving-Average draufrechnen

In [ ]:
ma_saison_list = add_saison(prediction_list_MA, seasonal_components)
#ma_saison_list

#### DataFrame-Erweiterung und plotten der Prognosewerte auf Basis MA + Saisonkomponente

In [ ]:
test_DF['Pred_MA_Sasion'] = ma_saison_list

In [ ]:
orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(test_DF.Quartal, test_DF.Pred_MA_Sasion, color='red', label='Pred')
plt.legend(loc='best')
plt.title('Original vs. Pred_MA')
plt.show()

### Bestimmtheitsmaß berechnen

In [ ]:
z_array = rev_DF.Revs.values.tolist()
z_array = z_array[3:len(z_array) - 3]

r = r_power(z_array, ma_saison_list)
r

## Prognose

In [ ]:
prediction_for_list2 = range(1,55 + 1)
prediction_list_MA2 = predict(a, b, prediction_for_list2, exp=True)
#prediction_list_MA2

In [ ]:
seasonal_components2 = seasonal_components[0:4] * 15
seasonal_components2 = seasonal_components2[0:len(prediction_list_MA2)]
#seasonal_components2

In [ ]:
ma_saison_list2 = add_saison(prediction_list_MA2, seasonal_components2)
ma_saison_list2[50:]

In [ ]:
#orig = plt.plot(rev_DF.Revs, color='blue', label='Original')
pred = plt.plot(prediction_for_list2, ma_saison_list2, color='red', label='Pred')
plt.legend(loc='best')
plt.title('Original vs. Pred_MA')
plt.show()